In [32]:
import pandas as pd
from py4j.java_gateway import JavaGateway
import psycopg2

In [33]:
home = "/Users/liviaalmada/NYDataset/Link Level Traffic Estimates/"
home = "/media/livia/DATA/Dropbox/TRABALHO/PESQUISA/Doutorado/Dados/NY-Data/"

In [34]:
graphnodes = pd.read_csv(home+"nodes.csv")

In [35]:
graphlinks = pd.read_csv(home+"/links.csv", delimiter=',')

In [36]:
graphnodes.head()

,node_id,is_complete,num_in_links,num_out_links,osm_traffic_controller,xcoord,ycoord,osm_changeset,birth_timestamp,death_timestamp,grid_region_id
0,42467266,t,3,3,null,-73.917419,40.660051,3218174,1405267167641,40000000000000,225
1,42467262,t,4,4,null,-73.918294,40.659484,3218174,1405267167641,40000000000000,225
2,42467259,t,3,3,null,-73.919089,40.658966,3218174,1405267167641,40000000000000,205
3,42467258,t,3,3,null,-73.919968,40.658398,3218174,1405267167641,40000000000000,205
4,42467250,t,2,2,null,-74.005260,40.600091,7948780,1405267167641,40000000000000,143


In [37]:
graphlinks.head()

,link_id,begin_node_id,end_node_id,begin_angle,end_angle,street_length,osm_name,osm_class,osm_way_id,startX,startY,endX,endY,osm_changeset,birth_timestamp,death_timestamp
0,1,103235840,103225947,-161.51,16.99,84.295,PalisadeAvenue,tertiary,223694294,-74.021807,40.884127,-74.021994,40.883384,16353009,1405267167641,40000000000000
1,2,42516422,42516427,175.46,-4.68,260.392,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932013,40.639616,19443546,1405267167641,40000000000000
2,3,42516422,42516418,-4.60,175.40,256.804,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932509,40.644252,19443546,1405267167641,40000000000000
3,4,103235530,103235525,-103.43,82.40,76.107,HarristownRoad,tertiary,11580026,-74.142573,40.948839,-74.143464,40.948718,14802365,1405267167641,40000000000000
4,5,42762376,42756156,-23.92,156.25,197.829,KingslandAvenue,residential,5698286,-73.844794,40.877882,-73.845749,40.879508,8219326,1405267167641,40000000000000


### Create Graph

In [78]:
gateway = JavaGateway() 

In [79]:
gateway.newGraph("ny", home+"graph")

In [80]:
graph = gateway.getGraph("ny");

In [64]:
nodesMap = { }


In [81]:
idn = 0
for n in graphnodes.values:
    gateway.addNode(n[0], n[5], n[6])
    nodesMap[n[0]] = idn
    idn = idn + 1

In [82]:
graph.getNumberOfNodes()

95581

In [48]:
print(nodesMap.get(105398165))

None


In [83]:
nodesMap = dict()
idn = 0
for (idx, row) in graphlinks.iterrows():
    
    if(nodesMap.get(row['begin_node_id']) is None):
        gateway.addNode( row['begin_node_id'] , row['startX'] , row['startY'])
        nodesMap[row['begin_node_id']] = idn
        idn = idn + 1
    if(nodesMap.get(row['end_node_id']) is None):
        gateway.addNode( row['end_node_id'] , row['endX'] , row['endY'])
        nodesMap[row['end_node_id']] = idn
        idn = idn + 1
    gateway.addEdge(nodesMap.get(row['begin_node_id']), nodesMap.get(row['end_node_id']), row['street_length'])
       

In [85]:
graph.getNumberOfEdges()

260855

In [86]:
graph.getNumberOfNodes()

192151

In [87]:
graphcosts = pd.read_csv(home+"/links.csv", delimiter=',')

In [88]:
graphcosts.head()

,link_id,begin_node_id,end_node_id,begin_angle,end_angle,street_length,osm_name,osm_class,osm_way_id,startX,startY,endX,endY,osm_changeset,birth_timestamp,death_timestamp
0,1,103235840,103225947,-161.51,16.99,84.295,PalisadeAvenue,tertiary,223694294,-74.021807,40.884127,-74.021994,40.883384,16353009,1405267167641,40000000000000
1,2,42516422,42516427,175.46,-4.68,260.392,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932013,40.639616,19443546,1405267167641,40000000000000
2,3,42516422,42516418,-4.60,175.40,256.804,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932509,40.644252,19443546,1405267167641,40000000000000
3,4,103235530,103235525,-103.43,82.40,76.107,HarristownRoad,tertiary,11580026,-74.142573,40.948839,-74.143464,40.948718,14802365,1405267167641,40000000000000
4,5,42762376,42756156,-23.92,156.25,197.829,KingslandAvenue,residential,5698286,-73.844794,40.877882,-73.845749,40.879508,8219326,1405267167641,40000000000000


In [53]:
grouped = graphcosts.groupby('begin_node_id')

In [54]:
grouped.ngroups

96435

## Generate graph costs

In [89]:
conn = psycopg2.connect(host="localhost", database="newyork", user="postgres")

In [90]:
curs = conn.cursor()

In [91]:
curs.execute("select * from gaussian_tt where weekday = 0 limit 100")

In [92]:
costs = curs.fetchall()

In [76]:
costs

[(0, 0, 0, 0.0, 6.75443325581395, 0, 3.62320745223019, 5.16117, 30.0, None),
 (0, 0, 0, 1.0, 7.94172093023256, 0, 4.90570495625238, 4.26861, 30.0, None),
 (0, 0, 0, 2.0, 7.98333046511628, 0, 3.42517712754407, 5.83561, 30.0, None),
 (0, 0, 0, 3.0, 8.94469627906977, 0, 4.66846809537856, 6.51491, 30.0, None),
 (0, 0, 0, 4.0, 8.64578162790698, 0, 0.460042202411547, 7.23471, 10.299, None),
 (0, 0, 0, 5.0, 10.6746609302326, 0, 3.02276369607222, 9.18258, 30.0, None),
 (0, 0, 0, 6.0, 11.6690397674419, 0, 2.87509199249427, 9.71831, 30.0, None),
 (0, 0, 0, 7.0, 10.7001141860465, 0, 0.453173571651, 9.28077, 11.8399, None),
 (0, 0, 0, 8.0, 10.0533413953488, 0, 3.12173120003209, 8.21407, 30.0, None),
 (0, 0, 0, 9.0, 9.0162219047619, 0, 3.30743580389936, 7.12906, 30.0, None),
 (0, 0, 0, 10.0, 8.41105046511628, 0, 3.93138094906834, 6.55898, 30.0, None),
 (0,
  0,
  0,
  11.0,
  7.00311348837209,
  0,
  0.572090339198362,
  4.65651,
  8.17966,
  None),
 (0, 0, 0, 12.0, 7.16052534883721, 0, 3.562041779

In [94]:
for cost in costs:
   gateway.addGaussianCost(nodesMap.get(cost[0]),nodesMap.get(cost[1]),cost[4], cost[6], cost[0])

Py4JError: An error occurred while calling t.addGaussianCost. Trace:
py4j.Py4JException: Method addGaussianCost([null, null, class java.lang.Double, class java.lang.Double, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:272)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)

